In [4]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
from flask import Flask, send_from_directory
import os

server = Flask(__name__)
app = dash.Dash(__name__, server=server, url_base_pathname='/')

# Assuming your HTML files are in the specified directory
directory = '/Users/tamasmakos/dev/k-monitor/network_analysis/network_visualizations'
html_files = [f for f in os.listdir(directory) if f.endswith('.html')]

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div([
        dcc.Dropdown(
            id='file-dropdown',
            options=[{'label': f, 'value': f} for f in html_files],
            value=html_files[0] if html_files else None
        ),
        dcc.Input(
            id='search-input',
            type='text',
            placeholder='Search node names...'
        ),
        html.Div(id='page-content')
    ])
])

@app.callback(
    Output('page-content', 'children'),
    [Input('file-dropdown', 'value'),
     Input('search-input', 'value')]
)
def display_page(selected_file, search_input):
    if selected_file:
        filtered_nodes = filter_nodes(selected_file, search_input)
        return html.Iframe(
            src=f'/reports/{selected_file}',
            style={'height': '500px', 'width': '100%'},
            title=filtered_nodes
        )
    return "Please select a report."

def filter_nodes(selected_file, search_input):
    # Implement your node filtering logic here
    # You can use the selected_file and search_input variables to filter the nodes
    # Return a list of filtered node names
    pass

# Serve HTML files
@server.route('/reports/<path:path>')
def serve_report(path):
    if path in html_files:
        return send_from_directory(directory, path)
    return '404'

if __name__ == '__main__':
    app.run_server(debug=True)
